In [1]:
# 필수 라이브러리 설치
!pip install pandas matplotlib seaborn wordcloud kiwipiepy scikit-learn pyarrow

Defaulting to user installation because normal site-packages is not writeable


# 뉴스 기사 통합 분석 리포트

이 노트북은 뉴스 기사의 여러 지표와 독자 인구 통계, 기사 본문 내용을 종합적으로 분석합니다.

**분석 파이프라인:**
1.  **기사 지표 분석**: 상위 기사의 핵심 지표 시각화
2.  **인구통계 분석**: 상위 기사의 독자층 분포 시각화
3.  **키워드 분석**: 워드클라우드를 통한 핵심 키워드 도출
4.  **토픽 모델링**: LDA를 활용한 기사 주제 군집화

In [1]:
# 라이브러리 Import
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from kiwipiepy import Kiwi
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import platform

In [18]:
# 공통 유틸리티 함수

"""
공통 유틸리티 함수 모듈
- 한글 폰트 설정
- 데이터 로드 최적화
- 상위 기사 추출
"""

import pandas as pd
import matplotlib.pyplot as plt
import platform
from sklearn.preprocessing import MinMaxScaler


def set_korean_font():
    """운영체제에 맞는 한글 폰트를 설정하고, 폰트 경로를 반환합니다."""
    font_name = ''
    try:
        if platform.system() == 'Windows':
            font_name = 'Malgun Gothic'
            plt.rc('font', family=font_name)
        elif platform.system() == 'Darwin':  # Mac OS
            font_name = 'AppleGothic'
            plt.rc('font', family=font_name)
        else:  # Linux
            font_name = 'NanumGothic'
            plt.rc('font', family=font_name)

        # 마이너스 기호 깨짐 방지
        plt.rcParams['axes.unicode_minus'] = False
        print(f"성공: '{font_name}' 폰트가 설정되었습니다.")
        return font_name
    except:
        print("한글 폰트를 찾을 수 없습니다. 그래프의 한글이 깨질 수 있습니다.")
        return None


def get_font_path():
    """워드클라우드용 폰트 경로를 반환합니다."""
    if platform.system() == 'Windows':
        return 'c:/Windows/Fonts/malgun.ttf'
    elif platform.system() == 'Darwin':
        return '/System/Library/Fonts/Supplemental/AppleGothic.ttf'
    else:
        return None


def load_optimized_data(excel_path):
    """
    Feather 파일이 있으면 우선 로드하고, 없으면 Excel 파일을 로드 후 Feather로 저장합니다.
    """
    feather_path = excel_path.replace('.xlsx', '.feather')
    try:
        df = pd.read_feather(feather_path)
        print(f"성능 최적화: '{feather_path}' 파일에서 데이터를 빠르게 로드했습니다.")
    except FileNotFoundError:
        print(f"최초 실행: '{excel_path}' 파일에서 데이터를 로드합니다. 다음 실행부터는 속도가 향상됩니다.")
        df = pd.read_excel(excel_path)
        # Feather 형식으로 저장 (빠른 로드를 위해)
        try:
            df.to_feather(feather_path)
        except:
            # Feather 저장 실패시 그냥 진행
            pass
    return df


def get_top_articles(metrics_df, top_n=20):
    """
    정규화된 종합 점수를 기준으로 상위 N개의 article_id 목록을 반환합니다.

    Parameters:
    -----------
    metrics_df : DataFrame
        article_metrics_monthly 데이터
    top_n : int
        추출할 상위 기사 개수

    Returns:
    --------
    list : 상위 N개 article_id 리스트
    """
    summary = metrics_df.groupby('article_id').agg({
        'comments': 'sum',
        'likes': 'sum',
        'views_total': 'sum'
    }).reset_index()

    scaler = MinMaxScaler()
    cols_to_scale = ['comments', 'likes', 'views_total']
    summary[cols_to_scale] = scaler.fit_transform(summary[cols_to_scale])

    # 종합 점수 계산 (댓글 50%, 좋아요 30%, 조회수 20%)
    summary['score'] = (summary['comments'] * 0.5 +
                        summary['likes'] * 0.3 +
                        summary['views_total'] * 0.2)

    top_articles = summary.sort_values(by='score', ascending=False).head(top_n)
    # 문자열로 변환하여 반환 (demographics 데이터와 일치시키기 위해)
    return top_articles['article_id'].astype(str).tolist()


def get_stopwords():
    """불용어 리스트를 반환합니다."""

    return [
        '이', '그', '저', '것', '수', '등', '및', '위해', '통해', '대한', '때문',
        '관련', '가장', '지난', '최근', '바로', '경우', '중', '속',
        '기자', '기사', '뉴스', '언론사', '언론', '취재', '사람', '우리', '생각',
        '콘텐츠', '미디어', '방송', '사회'
    ]


In [19]:
# NewsAnalyzer 클래스 정의

"""
뉴스 기사 통합 분석 스크립트
1. 기사 지표 분석 (조회수, 좋아요, 댓글, 종합점수)
2. 인구통계 분석 (연령대/성별)
3. 키워드 분석 (워드클라우드)
4. 토픽 모델링 (LDA)
"""

class NewsAnalyzer:
    """뉴스 기사 분석 클래스"""

    def __init__(self, data_dir='./data/'):
        """초기화 및 데이터 로드"""
        self.data_dir = data_dir
        self.result_dir = './result/'
        self.font_name = set_korean_font()
        self.font_prop = {'family': self.font_name} if self.font_name else {}
        self.top_n = 20  # 분석할 상위 기사 개수
        self.kiwi = Kiwi()

        if not os.path.exists(self.result_dir):
            os.makedirs(self.result_dir)
        
        # 데이터 로드
        self._load_data()

        # 상위 N개 기사 추출
        self.top_article_ids = get_top_articles(self.metrics_df, self.top_n)

    def _load_data(self):
        """데이터 파일 로드"""
        try:
            self.metrics_df = load_optimized_data(f'{self.data_dir}article_metrics_monthly.xlsx')
            self.contents_df = pd.read_excel(f'{self.data_dir}contents.xlsx')
            self.demo_part1 = load_optimized_data(f'{self.data_dir}demographics_part001.xlsx')
            self.demo_part2 = load_optimized_data(f'{self.data_dir}demographics_part002.xlsx')

            self.metrics_df['article_id'] = self.metrics_df['article_id'].astype(str)
            self.contents_df['article_id'] = self.contents_df['article_id'].astype(str)
            self.demo_part1['article_id'] = self.demo_part1['article_id'].astype(str)
            self.demo_part2['article_id'] = self.demo_part2['article_id'].astype(str)
            print("모든 데이터 파일이 성공적으로 로드되었습니다.\n")
        except FileNotFoundError as e:
            print(f"Error: 파일을 찾을 수 없습니다 - {e.filename}")
            exit()

    def analyze_metrics(self):
        """1. 기사 지표 분석 (원본 + 정규화)"""
        print("=" * 50)
        print("1. 기사 지표 분석 시작")
        print("=" * 50)

        # 기사 ID 기준으로 데이터 합산
        article_summary = self.metrics_df.groupby('article_id').agg({
            'comments': 'sum',
            'likes': 'sum',
            'views_total': 'sum'
        }).reset_index()

        # 원본 데이터 분석
        self._analyze_and_plot(article_summary, is_normalized=False)

        # 정규화 데이터 분석
        self._analyze_and_plot(article_summary, is_normalized=True)

        print("기사 지표 분석 완료\n")

    def _analyze_and_plot(self, data_df, is_normalized=False):
        """지표별 상위 N개 막대 그래프 생성"""
        # 데이터 복사본 생성
        summary_df = data_df.copy()

        # 정규화 옵션 처리
        if is_normalized:
            scaler = MinMaxScaler()
            cols_to_scale = ['comments', 'likes', 'views_total']
            summary_df[cols_to_scale] = scaler.fit_transform(summary_df[cols_to_scale])
            title_suffix = "(정규화 데이터)"
            xlabel_suffix = " (정규화)"
            filename_suffix = "normalized"
        else:
            title_suffix = "(원본 데이터)"
            xlabel_suffix = ""
            filename_suffix = "raw"

        # 가중치 점수 계산
        summary_df['score'] = (summary_df['comments'] * 0.5 +
                               summary_df['likes'] * 0.3 +
                               summary_df['views_total'] * 0.2)
        summary_df['article_id'] = summary_df['article_id'].astype(str)

        # 각 지표별 상위 N개 데이터 추출
        top_data = {
            'views': summary_df.sort_values(by='views_total', ascending=False).head(self.top_n),
            'likes': summary_df.sort_values(by='likes', ascending=False).head(self.top_n),
            'comments': summary_df.sort_values(by='comments', ascending=False).head(self.top_n),
            'score': summary_df.sort_values(by='score', ascending=False).head(self.top_n)
        }

        # 그래프 시각화
        fig, axes = plt.subplots(2, 2, figsize=(18, 12))
        fig.suptitle(f'신문과방송 기사 핵심 지표 상위 TOP {self.top_n} {title_suffix}',
                     fontsize=20, y=1.02)

        # 공통 속성을 딕셔너리로 관리
        plot_configs = {
            'views': {'ax': axes[0, 0], 'x': 'views_total', 'xlabel': f'총 조회수{xlabel_suffix}', 'palette': 'viridis'},
            'likes': {'ax': axes[0, 1], 'x': 'likes', 'xlabel': f'총 좋아요 수{xlabel_suffix}', 'palette': 'plasma'},
            'comments': {'ax': axes[1, 0], 'x': 'comments', 'xlabel': f'총 댓글 수{xlabel_suffix}', 'palette': 'magma'},
            'score': {'ax': axes[1, 1], 'x': 'score', 'xlabel': '종합 점수', 'palette': 'cividis'}
        }

        for key, config in plot_configs.items():
            data = top_data[key]
            sns.barplot(ax=config['ax'], x=config['x'], y='article_id', data=data,
                        order=data['article_id'], palette=config['palette'],
                        hue='article_id', legend=False)
            config['ax'].set_title(f'{config["xlabel"].split(" (")[0]} 상위 TOP {self.top_n}')
            config['ax'].set_xlabel(config['xlabel'])
            config['ax'].set_ylabel('기사 ID' if config['ax'] in [axes[0, 0], axes[1, 0]] else '')

        plt.tight_layout(rect=[0, 0, 1, 0.98])
        plt.savefig(f'{self.result_dir}article_metrics_summary_{filename_suffix}.png', dpi=300)
        plt.close()
        print(f"  - {self.result_dir}article_metrics_summary_{filename_suffix}.png 저장 완료")

    def analyze_demographics(self):
        """2. 인구통계 분석 (연령대/성별)"""
        print("=" * 50)
        print("2. 인구통계 분석 시작")
        print("=" * 50)

        # 인구통계 데이터 합치기 및 전처리
        demographics_df = pd.concat([self.demo_part1, self.demo_part2], ignore_index=True)
        demographics_df['article_id'] = demographics_df['article_id'].astype(str)

        # 상위 N개 기사의 인구통계 데이터만 필터링
        top_demographics = demographics_df[
            demographics_df['article_id'].isin(self.top_article_ids)
        ].copy()

        # 1) 누적 막대 그래프 (전체 포함)
        self._plot_demographics_bar(top_demographics.copy())

        # 2) 히트맵 (전체 포함)
        self._plot_demographics_heatmap(top_demographics.copy(), exclude_total=False)

        # 3) 히트맵 (전체 제외)
        self._plot_demographics_heatmap(top_demographics.copy(), exclude_total=True)

        print("인구통계 분석 완료\n")

    def _plot_demographics_bar(self, top_demographics):
        """인구통계 누적 막대 그래프"""
        top_demographics['demographic_group'] = (
                top_demographics['age_group'] + " " + top_demographics['gender']
        )

        demographics_summary = top_demographics.groupby(
            ['article_id', 'demographic_group']
        )['ratio'].mean().reset_index()

        # 데이터 피벗
        pivot_df = demographics_summary.pivot(
            index='article_id', columns='demographic_group', values='ratio'
        ).fillna(0)
        pivot_df = pivot_df.loc[self.top_article_ids]

        # 시각화
        plt.style.use('seaborn-v0_8-whitegrid')
        fig, ax = plt.subplots(figsize=(14, 12))

        colors = plt.get_cmap('tab10', len(pivot_df.columns))
        pivot_df.plot(kind='barh', stacked=True, ax=ax, color=colors.colors, width=0.8)

        ax.set_title(f'종합 점수 상위 {self.top_n}개 기사의 독자층 분포 (연령/성별)',
                     fontsize=18, pad=20, fontdict=self.font_prop)
        ax.set_xlabel('비율 (%)', fontsize=12, fontdict=self.font_prop)
        ax.set_ylabel('기사 ID (상위 순)', fontsize=12, fontdict=self.font_prop)
        ax.invert_yaxis()

        legend = ax.legend(title='독자 그룹', bbox_to_anchor=(1.02, 1),
                           loc='upper left', borderaxespad=0.)
        if self.font_name:
            plt.setp(legend.get_texts(), fontfamily=self.font_name)
            plt.setp(legend.get_title(), fontfamily=self.font_name)

        plt.tight_layout(rect=[0, 0, 0.85, 1])
        plt.savefig(f'{self.result_dir}top_articles_demographics.png', dpi=300)
        plt.close()
        print(f"  - {self.result_dir}top_articles_demographics.png 저장 완료")

    def _plot_demographics_heatmap(self, top_demographics, exclude_total=False):
        """인구통계 히트맵"""
        # 전체 제외 옵션
        if exclude_total:
            top_demographics = top_demographics[
                top_demographics['age_group'] != '전체'
                ].copy()
            filename = f'{self.result_dir}top_articles_demographics_heatmap_전체제외.png'
        else:
            filename = f'{self.result_dir}top_articles_demographics_heatmap.png'

        top_demographics['demographic_group'] = (
                top_demographics['age_group'] + " " + top_demographics['gender']
        )

        demographics_summary = top_demographics.groupby(
            ['article_id', 'demographic_group']
        )['ratio'].mean().reset_index()

        # 데이터 피벗
        pivot_df = demographics_summary.pivot(
            index='article_id', columns='demographic_group', values='ratio'
        ).fillna(0)
        pivot_df = pivot_df.loc[self.top_article_ids]

        # 히트맵 시각화
        fig, ax = plt.subplots(figsize=(20, 12))

        sns.heatmap(
            pivot_df,
            annot=True,
            fmt=".1f",
            cmap="viridis",
            linewidths=.5,
            annot_kws={"size": 8},
            ax=ax
        )

        ax.set_title(f'종합 점수 상위 {self.top_n}개 기사의 독자층 분포 히트맵', fontsize=20, pad=20, fontdict=self.font_prop)
        ax.set_xlabel('독자 그룹 (연령/성별)', fontsize=12, fontdict=self.font_prop)
        ax.set_ylabel('기사 ID (상위 순)', fontsize=12, fontdict=self.font_prop)

        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", fontfamily=self.font_name)
        plt.setp(ax.get_yticklabels(), rotation=0, fontfamily=self.font_name)

        plt.tight_layout()
        plt.savefig(filename, dpi=300)
        plt.close()
        print(f"  - {filename} 저장 완료")

    def analyze_wordcloud(self):
        """3. 워드클라우드 분석"""
        print("=" * 50)
        print("3. 워드클라우드 분석 시작")
        print("=" * 50)

        # 상위 N개 기사의 본문 내용 필터링
        top_contents_df = self.contents_df[
            self.contents_df['article_id'].isin(self.top_article_ids)
        ]

        # 모든 본문을 하나의 텍스트로 합치기
        combined_content = " ".join(top_contents_df['content'].astype(str))

        # 워드클라우드 생성
        font_path = get_font_path()
        if not font_path:
            print("워드클라우드를 생성하기 위한 한글 폰트 경로를 찾을 수 없습니다.")
            return

        try:
            wordcloud, top_keywords = self._generate_wordcloud_and_counts(
                combined_content, font_path, top_n=self.top_n
            )

            # 워드클라우드 시각화 및 저장
            plt.figure(figsize=(12, 10))
            plt.imshow(wordcloud, interpolation='bilinear')
            plt.title(f'종합 점수 상위 {self.top_n}개 기사 핵심 키워드', size=20, pad=20, fontdict=self.font_prop)
            plt.axis('off')
            plt.tight_layout(pad=0)
            plt.savefig(f'{self.result_dir}top_articles_wordcloud.png', dpi=300)
            plt.close()
            print(f"  - {self.result_dir}top_articles_wordcloud.png 저장 완료")

            print(f"\n--- 종합 점수 상위 {self.top_n}개 기사 핵심 키워드 TOP {self.top_n} ---")
            for i, (word, count) in enumerate(top_keywords, 1):
                print(f"{i:2d}위: {word} (언급 횟수: {count})")

        except Exception as e:
            print(f"워드클라우드 생성 중 오류 발생: {e}")
            print("konlpy와 Java(JDK)가 올바르게 설치되었는지 확인해주세요.")

        print("워드클라우드 분석 완료\n")

    def _generate_wordcloud_and_counts(self, text_data, font_path, top_n):
        """워드클라우드 객체와 상위 키워드 리스트 반환"""
        results = self.kiwi.analyze(text_data)
        nouns = [token.form for token in results[0][0] if token.tag in {'NNG', 'NNP'}]

        # 불용어 처리
        stopwords = get_stopwords()
        filtered_nouns = [word for word in nouns if len(word) > 1 and word not in stopwords]
        word_counts = Counter(filtered_nouns)

        wc = WordCloud(
            font_path=font_path,
            background_color='white',
            width=1000,
            height=800,
            max_words=100,
            colormap='viridis'
        ).generate_from_frequencies(word_counts)

        # 상위 N개 키워드 추출
        top_keywords = word_counts.most_common(top_n)

        return wc, top_keywords

    def analyze_topics_lda(self):
        """4. LDA 토픽 모델링 분석"""
        print("=" * 50)
        print("4. LDA 토픽 모델링 분석 시작")
        print("=" * 50)

        # 상위 N개 기사의 본문 내용 필터링
        top_contents_df = self.contents_df[
            self.contents_df['article_id'].isin(self.top_article_ids)
        ].copy()

        # 텍스트 전처리
        stopwords = get_stopwords()

        processed_docs = []
        for doc in top_contents_df['content'].astype(str):
            results = self.kiwi.analyze(doc)
            nouns = [token.form for token in results[0][0] if token.tag in {'NNG', 'NNP'}]

            filtered_nouns = [word for word in nouns if len(word) > 1 and word not in stopwords]
            processed_docs.append(" ".join(filtered_nouns))

        top_contents_df['processed_content'] = processed_docs

        # TF-IDF 벡터화
        vectorizer = TfidfVectorizer(max_df=0.85, min_df=2)
        doc_term_matrix = vectorizer.fit_transform(top_contents_df['processed_content'])

        # LDA 모델 학습
        NUM_TOPICS = 3
        lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, random_state=42)
        lda_model.fit(doc_term_matrix)

        # 결과 출력
        feature_names = vectorizer.get_feature_names_out()
        self._display_topics(lda_model, feature_names, 10)

        # 각 기사별 토픽 할당
        topic_distributions = lda_model.transform(doc_term_matrix)
        top_contents_df['topic_num'] = topic_distributions.argmax(axis=1)

        # 토픽별 기사 목록 출력
        print("\n--- 토픽별 기사 목록 ---")
        for i in range(NUM_TOPICS):
            topic_articles = top_contents_df[top_contents_df['topic_num'] == i]

            # 토픽 키워드
            topic_keywords = " ".join([
                feature_names[j] for j in lda_model.components_[i].argsort()[:-10 - 1:-1]
            ])
            print(f"\n[ 토픽 #{i + 1}: {topic_keywords} ]")

            if not topic_articles.empty:
                for index, row in topic_articles.iterrows():
                    article_id = row['article_id']
                    title = row['title']
                    print(f"  - (ID: {article_id}) {title}")
            else:
                print("  - 이 토픽에 해당하는 기사가 없습니다.")

        print("\nLDA 토픽 모델링 분석 완료\n")

    def _display_topics(self, model, feature_names, no_top_words):
        """LDA 모델의 토픽별 상위 단어를 출력"""
        print("\n--- LDA 토픽 모델링 결과 ---")
        for topic_idx, topic in enumerate(model.components_):
            top_words = " ".join([
                feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]
            ])
            print(f"토픽 #{topic_idx + 1}: {top_words}")

    def run_all_analyses(self):
        """모든 분석 실행"""
        print("\n" + "=" * 50)
        print("뉴스 기사 통합 분석 시작")
        print("=" * 50 + "\n")

        # 1. 기사 지표 분석
        self.analyze_metrics()

        # 2. 인구통계 분석
        self.analyze_demographics()

        # 3. 워드클라우드 분석
        self.analyze_wordcloud()

        # 4. LDA 토픽 모델링
        self.analyze_topics_lda()

        print("\n" + "=" * 50)
        print("모든 분석이 완료되었습니다!")
        print("=" * 50 + "\n")


def main():
    """메인 실행 함수"""
    analyzer = NewsAnalyzer(data_dir='./data/')

    # 전체 분석 실행
    analyzer.run_all_analyses()

    # 또는 개별 분석 실행 가능
    # analyzer.analyze_metrics()
    # analyzer.analyze_demographics()
    # analyzer.analyze_wordcloud()
    # analyzer.analyze_topics_lda()


In [20]:
# 분석기 실행 및 데이터 로드
data_dir = './data/' # 데이터 경로로 설정 !

analyzer = NewsAnalyzer(data_dir=data_dir)

성공: 'Malgun Gothic' 폰트가 설정되었습니다.
성능 최적화: './data/article_metrics_monthly.feather' 파일에서 데이터를 빠르게 로드했습니다.
성능 최적화: './data/demographics_part001.feather' 파일에서 데이터를 빠르게 로드했습니다.
성능 최적화: './data/demographics_part002.feather' 파일에서 데이터를 빠르게 로드했습니다.
모든 데이터 파일이 성공적으로 로드되었습니다.



In [21]:
# 1. 기사 지표 분석
analyzer.analyze_metrics()

1. 기사 지표 분석 시작
  - ./result/article_metrics_summary_raw.png 저장 완료
  - ./result/article_metrics_summary_normalized.png 저장 완료
기사 지표 분석 완료



In [22]:
# 2. 인구통계 분석
analyzer.analyze_demographics()

2. 인구통계 분석 시작
  - ./result/top_articles_demographics.png 저장 완료


C:\ProgramData\anaconda3\Lib\site-packages\seaborn\utils.py:61: UserWarning: Glyph 45224 (\N{HANGUL SYLLABLE NAM}) missing from current font.
  fig.canvas.draw()
C:\ProgramData\anaconda3\Lib\site-packages\seaborn\utils.py:61: UserWarning: Glyph 50668 (\N{HANGUL SYLLABLE YEO}) missing from current font.
  fig.canvas.draw()
C:\ProgramData\anaconda3\Lib\site-packages\seaborn\utils.py:61: UserWarning: Glyph 51204 (\N{HANGUL SYLLABLE JEON}) missing from current font.
  fig.canvas.draw()
C:\ProgramData\anaconda3\Lib\site-packages\seaborn\utils.py:61: UserWarning: Glyph 52404 (\N{HANGUL SYLLABLE CE}) missing from current font.
  fig.canvas.draw()


  - ./result/top_articles_demographics_heatmap.png 저장 완료


C:\ProgramData\anaconda3\Lib\site-packages\seaborn\utils.py:61: UserWarning: Glyph 45224 (\N{HANGUL SYLLABLE NAM}) missing from current font.
  fig.canvas.draw()
C:\ProgramData\anaconda3\Lib\site-packages\seaborn\utils.py:61: UserWarning: Glyph 50668 (\N{HANGUL SYLLABLE YEO}) missing from current font.
  fig.canvas.draw()


  - ./result/top_articles_demographics_heatmap_전체제외.png 저장 완료
인구통계 분석 완료



In [23]:
# 3. 키워드 분석 (워드클라우드)
analyzer.analyze_wordcloud()

3. 워드클라우드 분석 시작
  - ./result/top_articles_wordcloud.png 저장 완료

--- 종합 점수 상위 20개 기사 핵심 키워드 TOP 20 ---
 1위: 드라마 (언급 횟수: 131)
 2위: 이용 (언급 횟수: 122)
 3위: 숏폼 (언급 횟수: 120)
 4위: 서비스 (언급 횟수: 76)
 5위: 시장 (언급 횟수: 75)
 6위: 플랫폼 (언급 횟수: 71)
 7위: 소셜 (언급 횟수: 71)
 8위: 조사 (언급 횟수: 67)
 9위: 채널 (언급 횟수: 66)
10위: 세대 (언급 횟수: 64)
11위: 팩트 (언급 횟수: 58)
12위: 체크 (언급 횟수: 57)
13위: 보도 (언급 횟수: 52)
14위: 한국 (언급 횟수: 51)
15위: 신문 (언급 횟수: 51)
16위: 이용자 (언급 횟수: 50)
17위: 정보 (언급 횟수: 48)
18위: 교육 (언급 횟수: 48)
19위: 미국 (언급 횟수: 47)
20위: 중국 (언급 횟수: 46)
워드클라우드 분석 완료



In [24]:
# 4. 토픽 모델링 (LDA)
analyzer.analyze_topics_lda()

4. LDA 토픽 모델링 분석 시작

--- LDA 토픽 모델링 결과 ---
토픽 #1: 보도 경제 얼굴 이용 이벤트 명예 조사 참여 교육 자유
토픽 #2: 고통 세대 아이 넷플릭스 저널리즘 팀장 지망 문제 교육 포함
토픽 #3: 드라마 숏폼 채널 시장 소셜 연말 서비스 팟캐스트 플랫폼 연애

--- 토픽별 기사 목록 ---

[ 토픽 #1: 보도 경제 얼굴 이용 이벤트 명예 조사 참여 교육 자유 ]
  - (ID: 223338139549) [<2023 언론수용자 조사> 결과] 대부분 매체에서 뉴스 이용률 감소 언론·언론인에 대한 평가도 하락
  - (ID: 223367981164) [프랑스] 선출직 공무원  명예훼손 공소시효 연장,  언론 자유 침해일까?
  - (ID: 223428024936) 《신문과방송》 발간 60주년 기념 이벤트 🍀
  - (ID: 223430762932) [언론사 이탈하는 주니어 기자 실태] 한 명 한 명이 소중한 자원,  더는 회사 등지지 않도록
  - (ID: 223435080045) [팩트체크 현황 점검] ‘선거’에서 ‘탕후루’까지 아이템 선정이 ‘팩트체크’ 의 관건
  - (ID: 223498456595) [EBS 다큐프라임 <돈의 얼굴> 제작기] ‘경알못’을 위한 경제 교육 다큐멘터리

[ 토픽 #2: 고통 세대 아이 넷플릭스 저널리즘 팀장 지망 문제 교육 포함 ]
  - (ID: 223338149961) [MZ세대가 말하는 뉴스 이용법] “우리가 뉴스를 안 본다고?” 온라인 커뮤니티와 숏폼 통해 뉴스 본다
  - (ID: 223410633962) [북리뷰: 《한국의 기자》] 직업으로서의 기자, 어제와 오늘 그리고 미래
  - (ID: 223430759166) [언론사 채용, 현황과 문제점 진단] 논술로 뽑는 언론인? 취재와 기사 작성 능력이 먼저다
  - (ID: 223435090740) [북리뷰: 《고통 구경하는 사회》] 구경과 목격 사이 그 어딘가  뉴스의 현재 위치를 생각한다
  - (ID: 223468670006) 